In [ ]:

%load_ext autoreload
%autoreload 2

import helper as hlp
import json


c:\Users\dusan\OneDrive\Desktop\pp\plannerv2\pv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:


# # Define your structured output model:
# class StructuredOutput(BaseModel):
#     field1: str = Field(..., description="A man's name (required)")
#     person_number: int = Field(..., description="A person's age (required)")
#     decimal: float = Field(None, description="A decimal number of person height in cm (optional)")
#     things: List[str] = Field(None, description="A list of 3 cities (optional)")



# # Example usage
# if __name__ == "__main__":
#     # Example schema list
#     schema_list = [
#         {
#             'name': 'TaxToolz',
#             'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"The region of the user"},{"name":"income","type":"integer","required":true,"description":"The income of the user"},{"name":"deductions","type":"integer","required":true,"description":"The deductions of the user"}]'
#         },
#         {
#             'name': 'UntitledTool',
#             'parameters_json': '[]'
#         }
#     ]
    
#     # Example input JSON
#     input_json = """
#     json    {
#         "TaxToolz": {
#             "region2": "US",
#             "relevance_score": 85,
#             "income": 50000
#         },
#         "UntitledTool": {
#             "relevance_score": 25
#         }
#     }
#     """
    
#     # Process and validate
#     result = hlp.validate_and_restructure(input_json, schema_list)
#     print(json.dumps(result, indent=2))


# # ----- Usage Example -----
# system_prompt = "Generate a person's stats"
# user_message = "Let's go with your suggestion"

# # Example conversation history with proper format for the direct API approach
# message_history = [
#     {"role": "user", "content": "I need information about an old man from Serbia."},
#     {"role": "assistant", "content": "I'd be happy to help you generate information about a person. But let's make him a young man from German?"},
#     {"role": "user", "content": "OK. He likes visiting different cities and knows them well."},
#     {"role": "assistant", "content": "I understand this person likes visiting different cities and is familiar with cities. Would you like me to generate additional information about him?"}
# ]

# # Replace with your actual Google API key

# output_dict = generate_structured_analysis(
#     StructuredOutput,
#     user_message,
#     message_history,
#     google_api_key,
#     system_prompt
# )

# print("Structured Output:", output_dict.get("structured_output"))
# print("Missing Required Fields:", output_dict.get("missing_required_fields"))
# print("Missing Optional Fields:", output_dict.get("missing_optional_fields"))
# print("Populated Fields:", output_dict.get("populated_fields"))
# print("All Missing Fields:", output_dict.get("all_missing_fields"))

# if "error" in output_dict:
#     print("\nError:", output_dict.get("error"))
#     print("Raw Response:", output_dict.get("raw_response"))

{
  "TaxToolz": {
    "parameters": {
      "region2": "US",
      "income": 50000
    },
    "relevance_score": 85,
    "validation": {
      "relevance_score_error": false,
      "required_params_error": true,
      "missing_required_params": [
        "deductions"
      ],
      "missing_or_invalid_optional_params": [],
      "fields_found": [
        "relevance_score",
        "region2",
        "income"
      ],
      "error_string": "Required parameter 'deductions' is missing"
    }
  },
  "UntitledTool": {
    "parameters": {},
    "relevance_score": 25,
    "validation": {
      "relevance_score_error": false,
      "required_params_error": false,
      "missing_required_params": [],
      "missing_or_invalid_optional_params": [],
      "fields_found": [
        "relevance_score"
      ],
      "error_string": ""
    }
  }
}


In [131]:

import helper as hlp

# Load all configurations
configs = hlp.load_all_configs()

# Access individual configs
planner_config = configs["planner"]
user_data = configs["user_data"]  # This is the parsed user_json as a dict
tools = configs["tools"]
content_items = configs["content"]["items"]

# Use the configuration data...
planner_mode = planner_config.get("mode")
combined_result = hlp.generate_schema_for_tools(tools)
combined_result

'    \'TaxToolz\':{\n        \'region2\' = \'string\',# required / User region\n        \'age\' = \'integer\',# required / User exact age\n        \'relevance_score\' = \'integer\',# required / The relevance score of the "TaxToolz" tool based soley on the function/tool description (not its parameters) from 0 to 100 with 100 being the most relevant and 0 being the least relevant\n        \'required_fields_completed\' = \'integer\',# required / Score from 0-100 indicating that the extent that values imputed for the required fields fit the descriptions of the required fields provided. 0 being least fitting the parameter criteria and 100 perfectly fitting the criteria. Even though something can be added for the field, it is not the same thing as fitting the criteria perfectly, for example \'age\' field might be populated given that the user is a teen with a value from 13-19, but if the criteria is for \'age\' to be a value between 18-65, then the field is not fitting the criteria perfectly

In [92]:
tools

[{'name': 'TaxToolz',
  'description': 'Handles tax-related queries',
  'priority': 50,
  'display_mode': 'as-is',
  'index_name': 'payrollIndex',
  'top_k': 4,
  'reranker': 'top3',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"},{"name":"age","type":"integer","required":true,"description":"User exact age"}]',
  'user_fields_mapping': '{"age":"23"}',
  'disambiguation_level': 10,
  'can_be_overridden_when_sticky': False},
 {'name': 'Payroll',
  'description': 'Call this tool for payroll questions',
  'priority': 100,
  'display_mode': 'as-is',
  'index_name': 'sfss',
  'top_k': 12,
  'reranker': 'top1',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"}]',
  'user_fields_mapping': '{}',
  'disambiguation_level': 10,
  'can_be_overridden_when_sticky': True}]

In [60]:
planner_config

{'mode': 'smart',
 'schema_data': '{}',
 'disambiguation_threshold': 10,
 'relevance_threshold': 10}

In [98]:
message_history = []

In [119]:

user_message = """payrol teen from nevada"""
message_history.append({"role": "user", "content": user_message})

system_prompt = f"""You are a helpful product support chatbot for Intuit Quickbooks. You have access to the chat history with the user and you should provide output in strict JSON mode according to the following schema:

{combined_result}
"""

if planner_mode in ['fast','fast_listen_override']:
    model_name = 'gemini-2.0-flash'
elif planner_mode in ['smart']:
    model_name = 'gemini-2.0-pro-exp-02-05'
else:
    print("Invalid planner mode")


out_str = hlp.generate_flexible_structure(
    user_message = user_message,
    message_history = message_history,
    google_api_key = hlp.google_api_key,
    system_prompt = system_prompt,
    model_name  = model_name
)

validated_structure_out = hlp.validate_and_restructure(out_str[1], tools)

#message_history.append({"role": "assistant", "content": bot_reply})

In [120]:
validated_structure_out['Payroll']
validated_structure_out

{'TaxToolz': {'parameters': {'region2': 'Nevada', 'age': 16},
  'relevance_score': 10,
  'required_fields_completed': 60,
  'validation': {'relevance_score_error': False,
   'required_fields_completed_error': False,
   'required_params_error': False,
   'missing_required_params': [],
   'missing_or_invalid_optional_params': [],
   'fields_found': ['relevance_score',
    'required_fields_completed',
    'region2',
    'age'],
   'error_string': ''}},
 'Payroll': {'parameters': {'region2': 'Nevada'},
  'relevance_score': 95,
  'required_fields_completed': 100,
  'validation': {'relevance_score_error': False,
   'required_fields_completed_error': False,
   'required_params_error': False,
   'missing_required_params': [],
   'missing_or_invalid_optional_params': [],
   'fields_found': ['relevance_score', 'required_fields_completed', 'region2'],
   'error_string': ''}}}

In [96]:
tools

[{'name': 'TaxToolz',
  'description': 'Handles tax-related queries',
  'priority': 50,
  'display_mode': 'as-is',
  'index_name': 'payrollIndex',
  'top_k': 4,
  'reranker': 'top3',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"},{"name":"age","type":"integer","required":true,"description":"User exact age"}]',
  'user_fields_mapping': '{"age":"23"}',
  'disambiguation_level': 10,
  'can_be_overridden_when_sticky': False},
 {'name': 'Payroll',
  'description': 'Call this tool for payroll questions',
  'priority': 100,
  'display_mode': 'as-is',
  'index_name': 'sfss',
  'top_k': 12,
  'reranker': 'top1',
  'parameters_json': '[{"name":"region2","type":"string","required":true,"description":"User region"}]',
  'user_fields_mapping': '{}',
  'disambiguation_level': 10,
  'can_be_overridden_when_sticky': True}]

In [122]:
v.get("relevance_score")

95

In [ ]:
import json

successful_tools = []
need_disambig_tools = []

for k,v in validated_structure_out.items():
    if v.get("relevance_score") >= planner_config['relevance_threshold']:
        for t in tools:
            if t['name'] == k: 
                
                payload = {'name':k,'description':t['description'],'display_mode':t['display_mode'],'can_be_overridden_when_sticky':t['can_be_overridden_when_sticky'],'priority':t['priority']}

                if v.get('required_fields_completed') >= t['disambiguation_level']*10 and len(v.get('validation')['missing_required_params'])==0:
                    # all required fields are completed and disambiguation level is met
                    
                    payload['disambiguation_fields'] = []
                    successful_tools.append(payload)
                else: # disambiguation needed

                    parameters_dict = json.loads(t['parameters_json'])

                    if v.get('required_fields_completed') < t['disambiguation_level']*10: # add all fields to disambituation because threshold not met
                        
                        print(t['disambiguation_level']*10)
                        print(v.get('required_fields_completed'))

                        disambiguation_fields = []
                        for p in parameters_dict:
                            disambiguation_fields.append({p['name']:p['description']})
                    else: # threshold met, but some fields are missing so add them to disambiguation
                        disambiguation_fields = []
                        
                        for p in parameters_dict:
                            if p['name'] in v.get('validation')['missing_required_params']:
                                disambiguation_fields.append({p['name']:p['description']})
                                

                    payload['disambiguation_fields'] = disambiguation_fields
                    need_disambig_tools.append(payload)
                


        
def get_lowest_priority_items(items_list):
    if not items_list:
        return []
    
    # Find the minimum priority value
    min_priority = min(item['priority'] for item in items_list)
    
    # Filter the list to only include items with the minimum priority
    lowest_priority_items = [item for item in items_list if item['priority'] == min_priority]
    
    return lowest_priority_items


successful_tools_by_priority = get_lowest_priority_items(successful_tools)
need_disambig_tools_by_priority = get_lowest_priority_items(need_disambig_tools)
print(successful_tools_by_priority)
print("$$$$$$$$$$$$$$$$")
print(need_disambig_tools_by_priority)

# successful_tools,need_disambig_tools

# turn off planner disambiguation setting
# 

[{'name': 'TaxToolz', 'description': 'Handles tax-related queries', 'display_mode': 'as-is', 'can_be_overridden_when_sticky': False, 'priority': 10, 'disambiguation_fields': []}, {'name': 'Payroll', 'description': 'Call this tool for payroll questions', 'display_mode': 'as-is', 'can_be_overridden_when_sticky': True, 'priority': 10, 'disambiguation_fields': []}]
$$$$$$$$$$$$$$$$
[]


In [ ]:
# re-index all queries and content for an updated index as soon as it's updated (across content)
# Add secondary disambiguation threshold on tools
# rerank add tags to content for SDR
# turn off planner disambiguation setting
# if no execution needed but disambiguation needed, summarize disambig
# if no execution needed, have a default response
# if execution needed, find content under tool index (in sequence, if multiple tools) and filter using user parameters
# find content with top k for tool and rerank
# summarize if summarizable, even with multiple tools
# disambiguate against returened queries if below secondary disambiguation threshold and make tool sticky
# concatinate if not summarizable

In [ ]:
p['name'] in v.get('validation')['missing_required_params']

TypeError: string indices must be integers, not 'str'

In [103]:

# Process and validate
result = validate_and_restructure(out_str[1], tools)
print(json.dumps(result, indent=2))

NameError: name 'validate_and_restructure' is not defined